#### Data Extraction
extract essential information from JSON for the analysis
"title","views", "likes","comments", 'duration','tag','description', 'channel', 'category', 'published'

In [1]:
#extracting json to df
import os, json
import pandas as pd
import numpy as np
import glob
from datetime import datetime
pd.set_option('display.max_columns', None)

dir = 'data'
path = os.path.join(dir, '**/*.json')
file_list = glob.glob(path)
print('Total number of channels ' + str(len(file_list)))
# print(file_list)
dfs = list()

for file in file_list:
    with open(file, 'r') as f:
        data = json.load(f)
    channel_id, stats = data.popitem()
    pchannel_stats = stats["channel_statistics"]
    video_stats = stats["video_data"]
    # sorted_vids = sorted(video_stats.items(), key=lambda item: int(item[1]["viewCount"]),reverse=True)
    vids = video_stats.items()
    stats = []
    for vid in vids:
        video_id = vid[0]
        title = vid[1]["title"]
        try:
            views = vid[1]["viewCount"]
            likes = vid[1]["likeCount"]
            duration = vid[1]['duration']
            tags = vid[1]['tags']
            description = vid[1]['description']
            comments = vid[1]["commentCount"]
            channel = vid[1]['channelTitle']
            published = vid[1]['publishedAt'].split('T')[0]
        except:
            pass
        cat = os.path.dirname(file).split('\\')[1]
        stats.append([title,views, published, likes, comments, duration, tags, description, channel, cat])
    vid_df = pd.DataFrame(stats, columns=["title","views", 'published',"likes","comments", 'duration','tag','description', 'channel', 'category'])
    dfs.append(vid_df)
df = pd.concat(dfs, ignore_index=True)


Total number of channels 86


In [5]:
df.published = pd.to_datetime(df.published, format='%Y-%m-%d')
df.sample(5), df.shape

(                                                   title    views  published  \
 6771   Introduction to the unit circle | Trigonometry...  1596189 2012-12-10   
 16340  Marshmello Celebrates National Marshmallow Day...   586165 2019-08-31   
 17026  Breathe Carolina & Sunstars - DYSYLM (Official...  1438315 2018-01-05   
 8582        Dangerous Soaps: How Animals Use Surfactants   110039 2020-08-20   
 33331  Day 7 - Synchronize  |  BREATH - A 30 Day Yoga...  1728113 2021-01-08   
 
        likes comments  duration  \
 6771    7955       41    PT9M4S   
 16340  29455     1376   PT2M19S   
 17026  33253      738   PT2M27S   
 8582    6761      238   PT5M57S   
 33331  33508     3158  PT27M37S   
 
                                                      tag  \
 6771              [probability distribution, continuous]   
 16340  [marshmello, marshmallow, happier, here with m...   
 17026  [Spinnin', Records, Spinnin' Records, Breathe ...   
 8582   [SciShow, science, Hank, Green, education,

In [3]:
import pandas as pd
import gensim
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords

In [4]:
# set of english stopwords
stopset = set(stopwords.words('english'))

# grab the titles
titles = df.title

# create title list of string

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\poomk/nltk_data'
    - 'c:\\Users\\poomk\\miniconda3\\envs\\dataenv\\nltk_data'
    - 'c:\\Users\\poomk\\miniconda3\\envs\\dataenv\\share\\nltk_data'
    - 'c:\\Users\\poomk\\miniconda3\\envs\\dataenv\\lib\\nltk_data'
    - 'C:\\Users\\poomk\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
